In [1]:
%matplotlib inline
import pandas as pd
import nivapy3 as nivapy
import matplotlib.pyplot as plt
import matplotlib
import geopandas as gpd
import seaborn as sn
import numpy as np
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

plt.style.use("ggplot")

In [2]:
# Connect to Nivabase
eng = nivapy.da.connect()

Username:  ···
Password:  ········


Connection successful.


# Compare 1995 and 2019 survey results

This notebook generates a one-page summary plot for each parameter in the 1000 Lake survey.

## 1. Stations

Read stations and assign to forsuringsregionene.

In [3]:
# Get stations
stn_df = nivapy.da.select_resa_project_stations([4530], eng)

# Assign acid region IDs
shp_path = r"../data/forsuringsregionene.shp"
reg_gdf = gpd.read_file(shp_path)
stn_df = nivapy.spatial.identify_point_in_polygon(stn_df, reg_gdf, poly_col="name")

print(len(stn_df), "stations in project.")

stn_df.head()

1003 stations in project.


,station_id,station_code,station_name,latitude,longitude,altitude,name
0,3167,620-4-6,Ørteren,60.470,7.795,1147.0,Fjellregion - Sør-Norge
1,3168,621-1-27,Flåvatna,60.200,9.183,855.0,Fjellregion - Sør-Norge
2,3169,621-3-5,Soneren,60.061,9.545,104.0,Østlandet - Sør
3,3170,622-2-43,Trytetjern,60.213,9.764,275.0,Østlandet - Sør
4,3171,622-4-4,Krøderen,60.327,9.645,133.0,Østlandet - Sør


## 2. Get parameters measured in 2019

In [4]:
# Time period of interest
st_dt = "2019-08-01"
end_dt = "2019-12-31"

par_df = nivapy.da.select_resa_station_parameters(
    stn_df["station_id"].unique(), st_dt, end_dt, eng
)

par_df.head()

36 parameters available for the selected stations and dates.


,parameter_id,parameter_name,unit
0,3,ALK,mmol/l
1,50,Al,µg/l
2,55,Al/Il,µg/l
3,48,Al/R,µg/l
4,223,As,µg/l


## 3. Get water chemistry

Points to note:

 * This code does not specifically identify the "1000 lakes" samples - it just gets all samples from autumn 1995 and 2019 for the 1000 stations in the project. Some of these sites may be sampled mutliple times during the autumn as part of other monitoring programmes. At present, these values are just averaged
 
 * The 1995 samples are all recorded at `depth = 0`.  Some of the 2019 samples are deeper (often at `depth = 2 m`. I'm assuming that samples from deeper than 2 m are from other projects and therefore excluding them there

In [5]:
# Data from 1995
wc_95, dups_95 = nivapy.da.select_resa_water_chemistry(
    stn_df["station_id"].unique(),
    par_df,
    "1995-08-01",
    "1995-12-31",
    eng,
    drop_dups=True,
)

# Data from 2019
wc_19, dups_19 = nivapy.da.select_resa_water_chemistry(
    stn_df["station_id"].unique(), par_df, st_dt, end_dt, eng, drop_dups=True
)

# Only surface samples
wc_95 = wc_95.query("(depth1 <= 2) and (depth2 <= 2)")
wc_19 = wc_19.query("(depth1 <= 2) and (depth2 <= 2)")
del wc_95["depth1"], wc_95["depth2"], wc_19["depth1"], wc_19["depth2"]

# Calculate averages for stations with multiple samples
idx_cols = ["station_id", "station_code", "station_name"]
wc_95 = wc_95.groupby(idx_cols).mean().reset_index()
wc_19 = wc_19.groupby(idx_cols).mean().reset_index()

WARNING
The database contains unexpected duplicate values for some station-date-parameter combinations.
Only the most recent values will be used, but you should check the repeated values are not errors.
The duplicated entries are returned in a separate dataframe.



## 4. Plot data

In [6]:
def plot_points(
    df,
    par,
    ax,
    lat="latitude",
    lon="longitude",
    cmap="coolwarm",
    s=10,
    title=None,
    vmin=None,
    vmax=None,
    vmax_pct=0.95,
    cbar_ticks=None,
    cbar_labels=None,
):
    """ Plot Norwegian point data on specified axis.
    """

    if vmax and vmax_pct:
        raise ValueError("Please specify 'vmax' or 'vmax_pct', not both.")

    # Plot Norway
    shp = cartopy.io.shapereader.natural_earth(
        resolution="50m", category="cultural", name="admin_0_countries"
    )
    reader = cartopy.io.shapereader.Reader(shp)
    countries = reader.records()

    # Add Norway outline
    for country in countries:
        if country.attributes["NAME"] == "Norway":
            ax.add_geometries(
                [country.geometry],
                ccrs.PlateCarree(),  # CRS of Natural Earth data
                facecolor="none",
                edgecolor="black",
                linewidth=1,
                zorder=5,
            )

    if vmax_pct:
        vmax = df[par].describe(percentiles=[0.95,]).loc["95%"]

    # Add points using linear colour ramp from 0 to vmax
    pts = ax.scatter(
        df[lon].values,
        df[lat].values,
        c=df[par].values,
        cmap=cmap,
        vmin=vmin,
        vmax=vmax,
        s=s,
        zorder=5,
        edgecolors="none",
        transform=ccrs.PlateCarree(),
    )

    # Add axis for colourbar
    cax = inset_axes(
        ax,
        width="5%",
        height="30%",
        loc="lower left",
        bbox_to_anchor=(0.8, 0.05, 1, 1),
        bbox_transform=ax.transAxes,
        borderpad=0,
    )

    # Add colourbar
    cbar = fig.colorbar(pts, cax=cax, ticks=cbar_ticks)
    if cbar_labels:
        cbar.ax.set_yticklabels(cbar_labels)

    # Turn off border around subplot
    ax.outline_patch.set_edgecolor("white")

    if title:
        ax.set_title(title, fontsize=12, loc="left")

In [7]:
# Loop over parameters
for col in wc_19.columns:
    if col in wc_95.columns:
        if col not in idx_cols:
            par, unit = col.split("_")
            print("Processing:", par)

            # Get 2019 data
            df19 = wc_19[["station_id", col]].dropna()
            nvals = len(df19)

            # Join co-ords
            df19 = pd.merge(
                df19,
                stn_df[["station_id", "latitude", "longitude", "name"]],
                how="left",
                on="station_id",
            )

            df9519 = pd.merge(
                df19,
                wc_95[["station_id", col]],
                how="left",
                on="station_id",
                suffixes=["_19", "_95"],
            ).dropna()

            # Calc ratio 2019:1995
            df9519["ratio_2019:1995"] = df9519[col + "_19"] / df9519[col + "_95"]
            df9519["log(ratio_2019:1995)"] = np.log10(df9519["ratio_2019:1995"])

            # Melt to lon
            melt_df = df9519[["station_id", "name", col + "_95", col + "_19"]].copy()
            melt_df.columns = ["station_id", "Region", "1995", "2019"]
            melt_df = pd.melt(
                melt_df,
                id_vars=["station_id", "Region"],
                var_name="Year",
                value_name=f"{par} ({unit})",
            )

            # Define co-ord system
            crs = ccrs.AlbersEqualArea(
                central_longitude=15,
                central_latitude=65,
                false_easting=650000,
                false_northing=800000,
                standard_parallels=(55, 75),
            )

            # Setup plot template
            fig = plt.figure(figsize=(12, 15))
            gs = fig.add_gridspec(ncols=2, nrows=4)
            ax1 = fig.add_subplot(gs[0:2, 0], projection=crs)
            ax1.set_extent([0, 1300000, 0, 1600000], crs=crs)

            ax2 = fig.add_subplot(gs[0:2, 1:], projection=crs)
            ax2.set_extent([0, 1300000, 0, 1600000], crs=crs)

            ax3 = fig.add_subplot(gs[2, 0])
            ax4 = fig.add_subplot(gs[2, 1])
            ax5 = fig.add_subplot(gs[3, :])

            # Plot values in 2019
            plot_points(
                df19, par=col, ax=ax1, title=f"(a) {par} in 2019 ({unit}; n = {nvals})",
            )

            # Plot ratio of values 2019:1995
            plot_points(
                df9519,
                par="log(ratio_2019:1995)",
                ax=ax2,
                vmin=-1,
                vmax=1,
                vmax_pct=None,
                title="(b) log(Value 2019 / Value 1995)",
                cbar_ticks=[-1, -0.7, -0.3, 0, 0.3, 0.7, 1],
                cbar_labels=["÷10", "÷5", "÷2", "Equal", "x2", "x5", "x10"],
            )

            # Plot CDF
            # If IQR of data is zero (e.g. because most values are LOD), Seaborn fails to
            # select bandwidth for KDE. See https://github.com/mwaskom/seaborn/issues/1990
            # Set bandwidth for F in 1995 manually
            if par == "F":
                sn.distplot(
                    df9519[col + "_95"],
                    hist=False,
                    kde_kws={"cumulative": True, "bw": 0.1},
                    ax=ax3,
                    label="1995",
                )

            else:
                sn.distplot(
                    df9519[col + "_95"],
                    hist=False,
                    kde_kws={"cumulative": True},
                    ax=ax3,
                    label="1995",
                )

            sn.distplot(
                df9519[col + "_19"].values,
                hist=False,
                kde_kws={"cumulative": True},
                ax=ax3,
                label="2019",
            )

            ax3.set_title(
                "(c) Cumulative distribution functions", fontsize=12, loc="left"
            )
            ax3.set_xlabel(f"{par} ({unit})", fontsize=10)
            ax3.set_ylabel("Probability", fontsize=10)

            # Scatter plot
            ax4.plot(
                df9519[col + "_95"],
                df9519[col + "_19"],
                "ro",
                fillstyle="none",
                markeredgewidth=0.5,
            )

            ax4.plot(df9519[col + "_95"], df9519[col + "_95"], "k--", label="1:1 line")
            ax4.legend(loc="best")
            ax4.set_title("(d) Scatter plot", fontsize=12, loc="left")
            ax4.set_xlabel(f"{par} in 1995 ({unit})", fontsize=10)
            ax4.set_ylabel(f"{par} in 2019 ({unit})", fontsize=10)

            # Box plots
            box = sn.violinplot(
                data=melt_df, x="Region", y=f"{par} ({unit})", hue="Year", ax=ax5
            )
            box.set_xticklabels(box.get_xticklabels(), rotation=30, ha="right")
            ax5.set_title("(e) Violin plots", fontsize=12, loc="left")

            plt.suptitle(par, fontsize=20, y=0.93, fontweight="bold")
            plt.subplots_adjust(hspace=0.4)

            # Save
            png_path = f"../output/summary_plots/1000_lakes_2019_{par.replace('/', '-').lower()}.png"
            plt.savefig(png_path, dpi=200)
            plt.close()

Processing: ALK
Processing: Al/Il
Processing: Al/R
Processing: As


/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Processing: Ca
Processing: Cd
Processing: Cl
Processing: Co
Processing: Cr
Processing: Cu
Processing: F
Processing: Fe
Processing: KOND
Processing: K
Processing: Mg
Processing: Mn
Processing: NH4-N
Processing: NO3-N
Processing: Na
Processing: Ni
Processing: Pb
Processing: SO4
Processing: SiO2
Processing: TOC
Processing: TOTN
Processing: TOTP
Processing: Zn
Processing: pH
